# SILVER LAYER - MASCARAMENTO PII (LGPD)

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from datetime import datetime


# CONFIGURAÇÃO

In [0]:
STORAGE_ACCOUNT = "mystoacc"
SILVER_PATH = f"abfss://silver@{STORAGE_ACCOUNT}.dfs.core.windows.net/pii"

spark.sql("USE CATALOG hive_metastore")
spark.sql("USE healthcare_silver")

PROCESSING_TIMESTAMP = datetime.now()
PROCESSING_DATE = PROCESSING_TIMESTAMP.strftime("%Y-%m-%d")

print(f"Processamento Silver - Mascaramento PII iniciado: {PROCESSING_TIMESTAMP}")
print("ATENÇÃO: Dados sensíveis - LGPD")
print("="*80)


In [0]:
# FUNÇÕES AUXILIARES

In [0]:
def log_transformation(table_name: str, input_count: int, output_count: int):
    """Loga métricas de transformação"""
    print(f"\n{table_name}:")
    print(f"  Input: {input_count} registros")
    print(f"  Output com mascaramento: {output_count} registros")

# FUNÇÕES DE MASCARAMENTO

In [0]:
def mask_cpf(cpf_col: str) -> col:
    """
    Mascara CPF mantendo últimos 2 dígitos
    Exemplo: 12345678910 → ***.***.***-10
    """
    return concat(
        lit("***.***.**"),
        lit("-"),
        substring(col(cpf_col), -2, 2)
    )

def mask_phone(phone_col: str) -> col:
    """
    Mascara telefone mantendo DDD e últimos 4 dígitos
    Exemplo: 11987654321 → (11) 9****-4321
    """
    return when(
        col(phone_col).isNotNull(),
        concat(
            lit("("),
            substring(col(phone_col), 1, 2),
            lit(") "),
            substring(col(phone_col), 3, 1),
            lit("****-"),
            substring(col(phone_col), -4, 4)
        )
    ).otherwise(None)

def mask_email(email_col: str) -> col:
    """
    Mascara email mantendo primeira letra e domínio
    Exemplo: joao.silva@email.com → j***@email.com
    """
    return when(
        col(email_col).isNotNull(),
        concat(
            substring(col(email_col), 1, 1),
            lit("***@"),
            regexp_extract(col(email_col), r"@(.+)$", 1)
        )
    ).otherwise(None)

def mask_name(name_col: str) -> col:
    """
    Mascara nome mantendo primeiro e último nome
    Exemplo: João Pedro Silva Santos → João S.
    """
    return when(
        col(name_col).isNotNull(),
        concat(
            split(col(name_col), " ").getItem(0),
            lit(" "),
            substring(split(col(name_col), " ").getItem(size(split(col(name_col), " ")) - 1), 1, 1),
            lit(".")
        )
    ).otherwise(None)

def mask_crm(crm_col: str) -> col:
    """
    Mascara CRM mantendo últimos 3 dígitos
    Exemplo: 123456 → ***456
    """
    return concat(
        lit("***"),
        substring(col(crm_col), -3, 3)
    )

print("✓ Funções de mascaramento carregadas")

In [0]:
# PROCESSAR PACIENTE_IDENTIDADE

In [0]:
print("\n" + "="*60)
print("Processando: paciente_identidade")
print("="*60)

try:
    # Ler tabela do catálogo (não do path físico)
    df_paciente_pii = spark.table("healthcare_silver.paciente_identidade")
    
    input_count = df_paciente_pii.count()
    print(f"Registros lidos: {input_count}")
    
    # Verificar se já tem colunas mascaradas
    existing_columns = df_paciente_pii.columns
    has_masked = "cpf_masked" in existing_columns
    
    if has_masked:
        print("⚠ Colunas mascaradas já existem - atualizando valores...")
    
    df_paciente_masked = df_paciente_pii \
        .withColumn("cpf_masked", mask_cpf("cpf")) \
        .withColumn("nome_completo_masked", mask_name("nome_completo")) \
        .withColumn("email_masked", mask_email("email"))
    
    df_paciente_final = df_paciente_masked.select(
        "sk_paciente",
        "cpf",
        "cpf_masked",
        "nome_completo",
        "nome_completo_masked",
        "email",
        "email_masked",
        "ingestion_timestamp",
        "is_pii",
        "silver_processed_at",
        "silver_processing_date"
    )
    
    output_count = df_paciente_final.count()
    
    df_paciente_final.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("healthcare_silver.paciente_identidade")
    
    log_transformation("paciente_identidade", input_count, output_count)
    
    print("✓ Colunas mascaradas processadas:")
    print("  - cpf_masked")
    print("  - nome_completo_masked")
    print("  - email_masked")
    
    # Exemplo de saída
    print("\nExemplo de mascaramento:")
    df_paciente_final.select("cpf", "cpf_masked", "nome_completo", "nome_completo_masked").show(3, truncate=False)
    
    paciente_status = "SUCCESS"
    
except Exception as e:
    print(f"✗ ERRO: {str(e)}")
    paciente_status = "FAILED"

# PROCESSAR MEDICO_IDENTIDADE

In [0]:
print("\n" + "="*60)
print("Processando: medico_identidade")
print("="*60)

try:
    df_medico_pii = spark.table("healthcare_silver.medico_identidade")
    
    input_count = df_medico_pii.count()
    print(f"Registros lidos: {input_count}")
    
    existing_columns = df_medico_pii.columns
    has_masked = "nome_medico_masked" in existing_columns
    
    if has_masked:
        print("⚠ Colunas mascaradas já existem - atualizando valores...")
    
    df_medico_masked = df_medico_pii \
        .withColumn("nome_medico_masked", mask_name("nome_medico")) \
        .withColumn("crm_masked", mask_crm("crm"))

    df_medico_final = df_medico_masked.select(
        "sk_medico",
        "nome_medico",
        "nome_medico_masked",
        "crm",
        "crm_masked",
        "ingestion_timestamp",
        "is_pii",
        "silver_processed_at",
        "silver_processing_date"
    )
    
    output_count = df_medico_final.count()
    
    df_medico_final.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("healthcare_silver.medico_identidade")
    
    log_transformation("medico_identidade", input_count, output_count)
    
    print("✓ Colunas mascaradas processadas:")
    print("  - nome_medico_masked")
    print("  - crm_masked")
    
    print("\nExemplo de mascaramento:")
    df_medico_final.select("nome_medico", "nome_medico_masked", "crm", "crm_masked").show(3, truncate=False)
    
    medico_status = "SUCCESS"
    
except Exception as e:
    print(f"✗ ERRO: {str(e)}")
    medico_status = "FAILED"

# CRIAR VIEWS SEGURAS (SÓ DADOS MASCARADOS)

In [0]:
print("\n" + "="*60)
print("Criando views seguras (apenas dados mascarados)")
print("="*60)

try:
    df_check = spark.table("healthcare_silver.paciente_identidade")
    
    if "cpf_masked" in df_check.columns:
        spark.sql("""
        CREATE OR REPLACE VIEW healthcare_silver.v_paciente_identidade_masked AS
        SELECT
            sk_paciente,
            cpf_masked AS cpf,
            nome_completo_masked AS nome_completo,
            email_masked AS email,
            silver_processed_at,
            silver_processing_date
        FROM healthcare_silver.paciente_identidade
        """)
        
        print("✓ View criada: v_paciente_identidade_masked")
    else:
        print("⚠ Colunas mascaradas não encontradas em paciente_identidade - view não criada")
    
    # Verificar médicos
    df_check_medico = spark.table("healthcare_silver.medico_identidade")
    
    if "nome_medico_masked" in df_check_medico.columns:
        spark.sql("""
        CREATE OR REPLACE VIEW healthcare_silver.v_medico_identidade_masked AS
        SELECT
            sk_medico,
            nome_medico_masked AS nome_medico,
            crm_masked AS crm,
            silver_processed_at,
            silver_processing_date
        FROM healthcare_silver.medico_identidade
        """)
        
        print("✓ View criada: v_medico_identidade_masked")
    else:
        print("⚠ Colunas mascaradas não encontradas em medico_identidade - view não criada")
        
except Exception as e:
    print(f"⚠ Erro ao criar views: {str(e)}")
    print("Views serão criadas na próxima execução após mascaramento completo")

# SUMÁRIO DE EXECUÇÃO

In [0]:
import pandas as pd

results = [
    {"table": "paciente_identidade", "status": paciente_status},
    {"table": "medico_identidade", "status": medico_status}
]

results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("SUMÁRIO DE EXECUÇÃO - SILVER MASCARAMENTO PII")
print("="*80)
display(results_df)

total_tables = len(results)
success_count = len([r for r in results if r['status'] == 'SUCCESS'])
failed_count = len([r for r in results if r['status'] == 'FAILED'])

print(f"\nTabelas PII processadas: {success_count}/{total_tables}")

if failed_count > 0:
    print(f"⚠ ALERTA: {failed_count} tabelas PII falharam!")
else:
    print("\n✓ Todas as tabelas PII foram mascaradas com sucesso!")

print("\n" + "="*80)

In [0]:
%sql
select * from hive_metastore.healthcare_silver.paciente_identidade